![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Legal/8.Long_Span_Extraction.ipynb)

# Legal Long Span Extraction

# Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [3]:
from johnsnowlabs import * 
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
jsl.install()

👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare-2.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up if John Snow Labs home exists in /root/.johnsnowlabs this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare-2.json
Installing /root/.johnsnowlabs/py_installs/spark_ocr-4.1.0-py3-none-any.whl to /usr/bin/python3
Running: /usr/bin/python3 -m pip install /root/.johnsnowlabs/py_installs/spark_ocr-4.1.0-py3-none-any.whl
👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare-2.json
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-4.2.0-py3-none-any.whl to /usr/bin/python3
Running: /usr/bin/python3 -m pip install /root/.johnsnowlabs/py_installs/spark_nlp_jsl-4.2.0-py3-none-any.whl
Installing pyspark to /usr/bin/python3
Running: /usr/bin/python3 -m pip install pyspark
Installed 3 products:
🕶 Spark-OCR

# Start Spark Session

In [1]:
from johnsnowlabs import * 
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

DEBUG START!
👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare-2.json
👌 Launched cpu-Optimized JVM SparkSession with Jars for: 🚀Spark-NLP==4.2.0, 💊Spark-Healthcare==4.2.0, 🕶Spark-OCR==4.1.0, running on ⚡ PySpark==3.1.2


In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# if you want to start the session with custom params as in start function above
def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")
      
    return builder.getOrCreate()

#spark = start(SECRET)


# NER, Question Generation and Question Answering for Long-Span extraction

Legal documents are known to be very long. Although you can divide the docuuments into paragraphs or sections, and those into sentences, the resulted sentences are still long.

Let's take a look at this example:

`Buyer shall use such materials and supplies only in accordance with the present agreement`

Not, let's imagine we want to extract three entities:
1) The Subject (`Buyer`)
2) The Action (`shall use`)
3) The Object (what the Buyer shall use? - `such materials and supplies only in accordance with the present agreement`)

Although Subject and Action can be totally manageable by traditional NER, it usually struggles the longer the spans are. Trying to model the extraction of Object with a simple NER may result in word fading, when some of the initial or ending words fade into `O`.

We present in this notebook a solution for Long Span Extraction: Using an Automatic Question Generator and a Question Answering model to:
1) First, using NER, detect entities as the `Subject` and the `Action`. 

Example: `Buyer - SUBJECT`, `shall use - OBJECT`

2) Automatically generate a question to ask for the `Object`, using `Subject` and `Action`;

Example: `What shall the Buyer use?`

3) Use the question and the sentence to retrieve `Object`, without the limitations of traditional NER;

Example: `What shall the Buyer use? such materials and supplies only in accordance with the present agreement`

Last, but not least, it's very important to chose a domain-specific Question Answering model.


# Answering the question - `What?`
Let's suppose we have the sentence of the example:

`The Buyer shall use such materials and supplies only in accordance with the present agreement`

In Spark NLP for Legal, we have a trained NER model which is able to extract Subjects (`Buyer`) and Actions (`shall use`) of agreements / obligations with good accuracy.

It's also trained for extracting the `Object` using NER, but it's usage is limited due to the restrictions commented above.

Let's get SUBJECT and ACTION and automatically create a question with them.

In [3]:
text = """The Buyer shall use such materials and supplies only in accordance with the present agreement"""

In [4]:
import pandas as pd

documentAssembler = nlp.DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sparktokenizer = nlp.Tokenizer()\
  .setInputCols("document")\
  .setOutputCol("token")

tokenClassifier = legal.BertForTokenClassification.pretrained("legner_obligations", "en", "legal/models")\
  .setInputCols("token", "document")\
  .setOutputCol("label")\
  .setCaseSensitive(True)

nerconverter = nlp.NerConverter()\
  .setInputCols(["document", "token", "label"])\
  .setOutputCol("ner_chunk")

# setEntities1 says which entity from NER goes first in the question
# setEntities2 says which entity from NER goes second in the question
# setQuestionMark to True adds a '?' at the end of the sentence (after entity 2)
# To sum up, the pattern is     [QUESTIONPRONOUN] [ENTITY1] [ENTITY2] [QUESTIONMARK]
qagenerator = legal.NerQuestionGenerator()\
  .setInputCols(["ner_chunk"])\
  .setOutputCol("question")\
  .setQuestionMark(False)\
  .setQuestionPronoun("What")\
  .setEntities1(["OBLIGATION_SUBJECT"])\
  .setEntities2(["OBLIGATION_ACTION"])

qa =nlp.BertForQuestionAnswering.pretrained("legqa_bert_large","en", "legal/models") \
  .setInputCols(["question", "document"]) \
  .setOutputCol("answer") \
  .setCaseSensitive(True)
  
pipeline =  Pipeline(stages=[
  documentAssembler,
  sparktokenizer,
  tokenClassifier,
  nerconverter,
  qagenerator,
    qa
    ]
)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

res = p_model.transform(spark.createDataFrame([[text]]).toDF("text"))

legner_obligations download started this may take some time.
[OK!]
legqa_bert_large download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


In [5]:
res.select('question.result').show(truncate=False)

+-----------------------+
|result                 |
+-----------------------+
|[What Buyer shall use ]|
+-----------------------+



In [6]:
res.select('answer.result').show(truncate=False)

+-------------------------------------------------+
|result                                           |
+-------------------------------------------------+
|[The Buyer shall use such materials and supplies]|
+-------------------------------------------------+



Let's get 4 additional examples

In [7]:
texts = [
    """The Buyer shall use such materials and supplies only in accordance with the present agreement""",
    """The Provider will notify the Buyer about the release date""",
    """Amazon agrees to supply 1-year license without fees""",
    """The Supplier should ship the product in less than 1 month"""
]

pdf = pd.DataFrame(texts, columns = ["text"])
pdf

,text
0,The Buyer shall use such materials and supplie...
1,The Provider will notify the Buyer about the r...
2,Amazon agrees to supply 1-year license without...
3,The Supplier should ship the product in less t...


In [8]:
df = spark.createDataFrame(pdf)

In [9]:
res = p_model.transform(df)

In [10]:
res.select('question.result', 'answer.result').show(truncate=False)

+-------------------------------+-----------------------------------------------------------+
|result                         |result                                                     |
+-------------------------------+-----------------------------------------------------------+
|[What Buyer shall use ]        |[The Buyer shall use such materials and supplies]          |
|[What Provider will notify ]   |[The Provider will notify the Buyer about the release date]|
|[What Amazon agrees to supply ]|[1 - year license without fees]                            |
|[What Supplier should ship ]   |[The Supplier should ship the product in less than 1 month]|
+-------------------------------+-----------------------------------------------------------+



# Answering the question - `To whom?`

Let's try to get now the Indirect Object. That is, the recipient of an action. For example, to whom a supplier should send a shipment.

In [11]:
qagenerator = legal.NerQuestionGenerator()\
  .setInputCols(["ner_chunk"])\
  .setOutputCol("question")\
  .setQuestionMark(False)\
  .setQuestionPronoun("To whom")\
  .setEntities1(["OBLIGATION_ACTION"])\
  .setEntities2(["OBLIGATION_SUBJECT"])
  
pipeline =  Pipeline(stages=[
  documentAssembler,
  sparktokenizer,
  tokenClassifier,
  nerconverter,
  qagenerator,
    qa
    ]
)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

text = """The Provider shall send the shipment to the Buyer"""
res = p_model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [12]:
res.select('text', 'question.result', 'answer.result').show(truncate=False)

+-------------------------------------------------+------------------------------+-----------------------------------------------+
|text                                             |result                        |result                                         |
+-------------------------------------------------+------------------------------+-----------------------------------------------+
|The Provider shall send the shipment to the Buyer|[To whom shall send Provider ]|[Provider shall send the shipment to the Buyer]|
+-------------------------------------------------+------------------------------+-----------------------------------------------+



# Other clauses
This approach works very well also with other clauses and phrases, as temporal ones. Let's try to ask for the deadline of a contract

In [13]:
qagenerator = legal.NerQuestionGenerator()\
  .setInputCols(["ner_chunk"])\
  .setOutputCol("question")\
  .setQuestionMark(False)\
  .setQuestionPronoun("Before when")\
  .setEntities1(["OBLIGATION_ACTION"])\
  .setEntities2(["OBLIGATION_SUBJECT"])
  
pipeline =  Pipeline(stages=[
  documentAssembler,
  sparktokenizer,
  tokenClassifier,
  nerconverter,
  qagenerator,
    qa
    ]
)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

text = """The customer should sign the contract before May, 2023"""
res = p_model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [14]:
res.select('text', 'question.result', 'answer.result').show(truncate=False)

+------------------------------------------------------+-----------------------------------+-------------------+
|text                                                  |result                             |result             |
+------------------------------------------------------+-----------------------------------+-------------------+
|The customer should sign the contract before May, 2023|[Before when should sign customer ]|[before May , 2023]|
+------------------------------------------------------+-----------------------------------+-------------------+

